<a id='top'></a>

# Watershed and Catchment Landcover Metrics

## Aspect and Percent North Covariates
Calculate aspect grid with the DEM used to create the flow network for each processing area (e.g. HUC8 in NHDPlus or
 region for BB, PWS, or Kodiak). We need the DEMs used for each flow direction grid, or we can't calculate covariates at
 the watershed scale.
 * **~~aspect_rch = calculate mean aspect for stream reach (zonal statistics)~~**
 * **~~aspect_cat = calculate mean aspect over catchments (zonal statistics)~~**
 * **north_wtd = create 1/0 grid of north facing cells from aspect grid (north = aspects from 315-45 degrees), ~~~use
  weighted flow accumulation to sum north facing cells, divide by flow accumulation grid to get % of north facing
  cells in each watershed~~~ Use tabulate area to quantify area of North cells (VALUE_1) and non-North (VALUE_0) cells
  for each watershed and calculate percent north as north_wtd = row[VALUE_1]/(row[VALUE_1]+row[VALUE_0])*100 .**
## Elevation Metrics
Calculate elevation metrics for catchment/watershed with temperature data using zonal statistics as table.
* <b> Running "ALL" zonal statistics as it does not allow you to choose two types of statistics
(ex statistics_type = ["MIN","MAX"]) instead set statistics_type="ALL"</b>
### Catchment Elevation Metrics
* **cat_elev_mn = mean elevation for catchment**
* **cat_elev_min = minimum elevation for catchment**
* **cat_elev_max = max elevation for catchment**
* **cat_elev_std = standard deviation of elevation for catchment**
### Watershed Elevation Metrics
* **wtd_elev_mn = mean watershed elevation**
* **wtd_elev_min = min watershed elevation**
* **wtd_elev_max = max watershed elevation**
* **wtd_elev_sd (or cv) = standard deviation of watershed elevation**
## Slope Metrics
Calculate slope metrics for catchment/watershed with temperature data using zonal statistics as table and
statistics_type = "ALL".
### Catchment Slope Metrics
* **cat_slope_mn = mean slope for catchment**
* **cat_slope_min = minimum slope for catchment**
* **cat_slope_max = max slope for catchment**
* **cat_slope_std = standard deviation of slope for catchment**
### Watershed Elevation Metrics
* **wtd_slope_mn = mean watershed slope**
* **wtd_slope_min = min watershed slope**
* **wtd_slope_max = max watershed slope**
* **wtd_slope_sd (or cv) = standard deviation of watershed slope**
## Lake, Wetland and Glacier Cover
Lake/Pond type waterbodies from NHDPLus hydrography for those regions with NHDPlus derived datasets and NHD waterbodies
for those regions with TauDEM derived waterbodies.  Merge all waterbodies together (FTYPE = 390) and use tabulate area
calculate percent cover of lakes in catchments/watersheds.
 * **Created local copies of NHDPlus and NHD waterbodies and exported to T driver here <>  **
Use NLCD grid (wetlands.tif) and tabulate area on wetlands grid with watersheds (sum) / divide by number of cells in
watershed from fac grid, wetlands from NLCD
## Distance from coast


## Import modules

In [1]:
import arcpy
import os
import datetime
import time
import sys
arcpy.env.overwriteOutput = True
today = datetime.datetime.now()
# Make the time stamp.
time_stamp = '{:%d%m%Y}'.format(today)
print(time_stamp)

path = os.getcwd()
print (path)
print (sys.base_exec_prefix)

20102021
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\landcover
C:\Users\dwmerrigan\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3


## Collect data and set working output locations
Set data_dir to folder containing all AKSSF regional subfolders/geoadatabases
* RS data folder: data_dir = r"W:\GIS\AKSSF"
* DM data folder: data_dir = r"D:\GIS_Temp\AKSSF_BeckyCopy\AKSSF"

In [2]:
# Set AKSSF Data directory
# data_dir = r"W:\GIS\AKSSF"

# dm local
data_dir = r"D:\GIS_Temp\AKSSF"

arcpy.env.workspace = data_dir
regions = arcpy.ListWorkspaces(workspace_type="Folder")
rois= []
print (regions)

# Path to create output folder/gdb
temppath = r"D:\GIS_temp" # Output folder
dirname = 'AKSSF_land_met'
tempgdbname = 'AKSSF_land_met.gdb'
temp_dir = os.path.join(temppath, dirname)

# Create temporary working gdb
if not arcpy.Exists(temp_dir):
    os.makedirs(temp_dir)
else:
    print('Working Folder already created...', temp_dir)

outcheck = os.path.join(temp_dir, tempgdbname)

if arcpy.Exists(outcheck):
    print ('Output location already exists', outcheck)
    outgdb = outcheck
if not arcpy.Exists(outcheck):
    print('Creating output GDB')
    tempgdb = arcpy.CreateFileGDB_management(temp_dir,tempgdbname)
    print ('Output geodatabase created at', outcheck)
    outgdb = tempgdb.getOutput(0)

['D:\\GIS_temp\\AKSSF\\Bristol_Bay', 'D:\\GIS_temp\\AKSSF\\Cook_Inlet', 'D:\\GIS_temp\\AKSSF\\Copper_River', 'D:\\GIS_temp\\AKSSF\\Kodiak', 'D:\\GIS_temp\\AKSSF\\Prince_William_Sound']
Working Folder already created... D:\GIS_temp\AKSSF_land_met
Output location already exists D:\GIS_temp\AKSSF_land_met\AKSSF_land_met.gdb


In [139]:
# Set regional workspaces from AKSSF data folder and store in list
arcpy.env.workspace = data_dir
regions  = arcpy.ListWorkspaces(workspace_type="Folder")
regions

['D:\\GIS_temp\\AKSSF\\Bristol_Bay',
 'D:\\GIS_temp\\AKSSF\\Cook_Inlet',
 'D:\\GIS_temp\\AKSSF\\Copper_River',
 'D:\\GIS_temp\\AKSSF\\Kodiak',
 'D:\\GIS_temp\\AKSSF\\Prince_William_Sound']

## Create aspect and north grids if they do not already exists
### NHDPlus

In [140]:
# Code to create merged elevation grid, aspect and north aspect grids for entire AKSSF region with NHDPlus elev data

# elrasters = []
# nhd_dat = r"C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\AKSSF_NHDPlus\\extracts"
# for dirpath, dirname, filenames in arcpy.da.Walk(nhd_dat, 'RasterDataset'):
#     for filename in filenames:
#         if filename == "elev_cm.tif":
#             elras = os.path.join(dirpath, filename)
#             elrasters.append(elras)
#             print (f'Appending {elras} to list')
#
# spatial_ref = arcpy.Describe(elrasters[0]).spatialReference
# print (spatial_ref.name)
# # Merge all nhdplus elevation rasters together
# elevname = 'AKSSF_NHDPlus_elev_cm.tif'
# AKSSF_elev_cm = arcpy.MosaicToNewRaster_management(elrasters,temp_dir,elevname, spatial_ref, "32_BIT_SIGNED", "5", "1", "LAST","FIRST")
#
# #make aspect raster for all of akssf from merged nhdplus
# from arcpy.sa import *
# aspname = 'AKSSF_NHDPlus_aspect.tif'
# asp_rast = Aspect(in_raster= AKSSF_elev_cm, method='Planar')
# asp_rast.save(os.path.join(temp_dir, aspname))
#
# # make north raster from aspect raster
# norname = 'AKSSF_NHDPlus_north_aspect.tif'
# aspect_in = Raster(asp_rast)
# north_raster = Con((aspect_in>=0)&(aspect_in<=45),1,Con((aspect_in<=360)&(aspect_in>=315),1,0))
# north_raster.save(os.path.join(temp_dir, norname))

### TauDEM - Create grids using Timms Composite DEM for entire TauDEM processed areas

In [141]:
# Code to create merged elevation grid, aspect and north aspect grids for entire AKSSF region with TauDEM elev data
# compDEM = r"C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\data\topography\AKSSF_Elevation_Composite_10m.tif"
# clip to study area
# AKSSF_elev_10m = r"D:\\GIS_temp\\AKSSF_Composite_10m_extract.tif"
#
# #make aspect raster for all of akssf from merged nhdplus
# from arcpy.sa import *
# aspname = 'AKSSF_Composite_10m_aspect.tif'
# asp_rast = Aspect(in_raster= AKSSF_elev_10m, method='Planar')
# asp_rast.save(os.path.join(temp_dir, aspname))
#
# # make north raster from aspect raster
# norname = 'AKSSF_Composite_10m_north_aspect.tif'
# aspect_in = Raster(asp_rast)
# north_raster = Con((aspect_in>=0)&(aspect_in<=45),1,Con((aspect_in<=360)&(aspect_in>=315),1,0))
# north_raster.save(os.path.join(temp_dir, norname))


## Begin GIS portion
- May be faster and save more space to create aspect rasters here using extract by mask?
- May be able to use aspect raster with tabulate intersection to calculate percent North (aspects from 315-45 degrees)

### <b>UPDATE - 2021-10-12 Do not run aspect metrics at this time. Only need to run percent North for watersheds</b>

In [197]:
from arcpy.sa import *
import re

# Set data_dir equal to folder containing AKSSF regional subfolders containing GDBs and raster datasets
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
regions  = arcpy.ListWorkspaces(workspace_type="Folder")

# Lists for variables not needed at present time
#cat_asp_ztables = []
#wtd_asp_ztables = []
#cat_pernorth_taba_tables=[]

# Lists to store output tables
wtd_pernorth_taba_tables=[]
cat_elev_ztables = []
wtd_elev_ztables = []
cat_slope_ztables = []
wtd_slope_ztables = []


# Clear lists
cat_cur_fields = []
wtd_cur_fields = []

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

# Seperate data by
nhdplus_dat = ['Cook_Inlet','Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

# Loop through all processing areas
rois = nhdplus_dat + tauDem_dat
# Test on Kodiak
rois = ['Kodiak']
for roi in rois:
    # Loop through regional folders
    for region in regions:
        if roi in str(region):
            print(f'{roi} using data from {region} folder')
            # Set data and variables unique to regions with NHDPlus Data
            if roi in nhdplus_dat:
                # # NHDPLus raster data for DM pc, variables will be overwritten below if they exist in source folder.
                # asp_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_aspect.tif"
                # elev_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_elev_cm.tif"
                # nor_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_north_aspect.tif"
                lakes_fc = r""
                # Fields for update cursor
                cat_cur_fields = ['cat_ID_txt', 'NHDPlusID',"cat_ID_con"]
                wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
                print (f'{roi} in {nhdplus_dat} AKSSF list, using cat_fields {cat_cur_fields} and watershed fields {wtd_cur_fields}')
                print(f'NHD data: north raster - {nor_rast}')
                print(f'         aspect raster - {asp_rast}')
                print(f'      elevation raster - {elev_rast}')
                print('----------')
            # Set data and variables unique to regions with TauDEM Data
            elif roi in tauDem_dat:
                # # TauDEM raster data for DM pc, variables will be overwritten below if they exist in source folder.
                # nor_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_Composite_10m_north_aspect.tif"
                # asp_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_Composite_10m_aspect.tif"
                # elev_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_Composite_10m_extract.tif"
                
                lakes_fc = r""
                # Fields for update cursor
                cat_cur_fields = ['cat_ID_txt', 'gridcode',"cat_ID_con"]
                wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]

                print (f'{roi} in {tauDem_dat} TauDEM list, using cat_fields {cat_cur_fields} and watershed fields {wtd_cur_fields}')
                print(f'TauDem data: north raster - {nor_rast}')
                print(f'         aspect raster - {asp_rast}')
                print(f'      elevation raster - {elev_rast}')
                print('----------')

            wtds = []
            cats = []
            # Start iter timing function
            iteration_start = time.time()
            # Set workspace to region folder
            arcpy.env.workspace = region
            gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
            walk = arcpy.da.Walk(region, datatype = ['FeatureClass','RasterDataset'])
            for dirpath, dirnames, filenames in walk:
                # Create list of watersheds
                for filename in filenames:
                    if (("wtd" in filename) and ("merge" not in filename)):
                        print(f" {filename} watershed identified")
                        wtds.append(os.path.join(dirpath, filename))
                    # Set watershed merged watersheds dataset
                    if 'wtds_merge'in filename:
                        wtd_merge = os.path.join(dirpath, filename)
                        print(f'Merged watershed dataset {filename} found')
                        print('----------')
                        wtdfieldnames = []
                        wtdlstFields = arcpy.ListFields(wtd_merge)
                        for field in wtdlstFields:
                            wtdfieldnames.append(field.name)
                        if str(wtd_cur_fields[0]) in wtdfieldnames:
                            print (f'{wtd_cur_fields[0]} field already in dataset')
                            print('----------')
                        else:
                            print (f'Adding {wtd_cur_fields[0]} field to watershed dataset {wtd_merge}')
                            print('----------')
                            # add cat_ID_txt field and concat cat_ID + region
                            arcpy.AddField_management(wtd_merge, str(wtd_cur_fields[0]),field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(wtd_merge, wtd_cur_fields[0:2]) as cur:
                                for row in cur:
                                    int_val = int(row[1])
                                    row[0] = str(int_val)
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)
                        if str(wtd_cur_fields[2]) in wtdfieldnames:
                            print (f'{wtd_cur_fields[2]} field already in dataset {wtd_merge}')
                            print('----------')
                        else:
                            print (f'Adding {wtd_cur_fields[2]} field to watershed dataset {wtd_merge}')
                            print('----------')
                            # add cat_ID_con field and concat cat_ID + region
                            arcpy.AddField_management(wtd_merge, str(wtd_cur_fields[2]),field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(wtd_merge, wtd_cur_fields) as cur:
                                for row in cur:
                                    int_val = int(row[1])
                                    row[2] = str(roi) +'_'+ str(int_val)
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)

                    # Select elevation raster
                    elif 'elev.tif' == filename:
                        elev_rast = os.path.join(dirpath, filename)

                    # # Select aspect raster
                    # elif 'aspect' in filename:
                    #     asp_rast = os.path.join(dirpath, filename)

                    # Select north raster
                    elif 'north.tif' == filename:
                        nor_rast = os.path.join(dirpath, filename)

                    # Select slope
                    elif 'slope.tif' == filename:
                        slope_rast = os.path.join(dirpath, filename)

                    # Select catch_int fc (catchments of interest for region) and make a copy
                    elif 'cats_intersect' in filename:
                        cats = os.path.join(dirpath,filename)
                        catlstfields = arcpy.ListFields(cats)
                        catfieldnames = []
                        for field in catlstfields:
                            catfieldnames.append(field.name)
                        if str(cat_cur_fields[0]) in catfieldnames:
                            print (f'{cat_cur_fields[0]} field already in dataset {cats}')
                            print('----------')
                        else:
                            print (f'Adding {cat_cur_fields[0]} field to catchment dataset {cats}')
                            print('----------')
                            # add cat_ID_txt field
                            arcpy.AddField_management(cats, str(cat_cur_fields[0]), field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(cats, cat_cur_fields[0:2]) as cur:
                                for row in cur:
                                    int_val = int(row[1])
                                    row[0] = str(int_val)
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)
                        if str(cat_cur_fields[2]) in catfieldnames:
                            print (f'{cat_cur_fields[2]} field already in dataset {cats}')
                            print('----------')
                        else:
                            print (f'Adding {cat_cur_fields[2]} field to catchment dataset {cats}')
                            print('----------')
                            # add cat_ID_txt field & cat_ID + region concat field
                            arcpy.AddField_management(cats,str(cat_cur_fields[2]),field_type='TEXT')
                            # populate cat_ID_con
                            with arcpy.da.UpdateCursor(cats, cat_cur_fields) as cur:
                                for row in cur:
                                    int_val = int(row[1])
                                    row[2] = str(roi) +'_'+ str(int_val)
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)

            print (f'Calculating topographic metrics for catchments & watersheds of interest in {roi}'
                   f' region')
            print ('----------')
            print(f'Geodatabase: {gdb}')
            print ('----------')
            print (f'Elevation Raster: {elev_rast}')
            print ('----------')
            print (f'North Aspect Raster: {nor_rast}')
            print ('----------')
            print (f'Slope Raster: {slope_rast}')
            print ('----------')        
            print (f'{len(wtds)} Watersheds to process')
            print ('----------')
            print (f'Catchment intersect {cats} selected')
            print ('----------')


            try:
                # Set output names/paths for watershed and catchment tables
                wtd_merge_name = roi + "_Watersheds_Merge"
                wtd_merge_path = os.path.join(outgdb,wtd_merge_name)

                # # Aspect variables
                # wtd_merge_asp_table_name = roi + "_Watersheds_Merge_AspectZstats"
                # wtd_merge_asp_table_path = os.path.join(outgdb, wtd_merge_asp_table_name)
                # cat_asp_table_name = roi + "_Catchments_AspectZstats"
                # cat_asp_table_path = os.path.join(outgdb, cat_asp_table_name)

                # Percent North variables
                wtd_merge_pernorth_table_name = roi + "_Watersheds_Merge_PercentNorth"
                wtd_merge_pernorth_table_path = os.path.join(outgdb, wtd_merge_pernorth_table_name)
                # cat_pernorth_table_name = roi + "_Catchments_PercentNorth"
                # cat_pernorth_table_path = os.path.join(outgdb, cat_pernorth_table_name)

                # Elevation variables
                wtd_merge_elev_table_name = roi + "_Watersheds_Merge_ElevZstats"
                wtd_merge_elev_table_path = os.path.join(outgdb, wtd_merge_elev_table_name)
                cat_elev_table_name = roi + "_Catchments_ElevZstats"
                cat_elev_table_path = os.path.join(outgdb, cat_elev_table_name)
                
                # Slope variables
                wtd_merge_slope_table_name = roi + "_Watershed_Merge_SlopeZstats"
                wtd_merge_slope_table_path = os.path.join(outgdb, wtd_merge_slope_table_name)
                cat_slope_table_name = roi + "_Catchments_SlopeZstats"
                cat_slope_table_path = os.path.join(outgdb, cat_slope_table_name)
                
                

                # # Watersheds already merged so no need to run at this time
                # if not arcpy.Exists(wtd_merge):
                #     mergestart = time.time()
                #     # Merge watersheds
                #     wtd_merge = arcpy.Merge_management( wtds, wtd_merge_path ,add_source='ADD_SOURCE_INFO')
                #     # Add wtd_id field
                #     arcpy.AddField_management(wtd_merge,'cat_ID_txt',field_type='TEXT')
                #     arcpy.AddField_management(wtd_merge,"cat_ID_con",field_type='TEXT')
                #     # Add region field
                #     arcpy.AddField_management(wtd_merge,'region',field_type='TEXT')
                #     # Populate watershed id and region information using update cursor - faster than field calc
                #     with arcpy.da.UpdateCursor(wtd_merge,['MERGE_SRC','cat_ID_txt','region','cat_ID_con']) as cur:
                #         for row in cur:
                #             row[1] = re.findall('\d+', row[0])[0]
                #             row[2] = roi
                #             row[3] = + str(roi) + '_'+ str(re.findall('\d+', row[0])[0])
                #             # Update
                #             cur.updateRow(row)
                #         del(row)
                #     del(cur)
                #     mergestop = time.time()
                #     mergetime = int (mergestop - mergestart)
                #     print(f'Watershed Merge for {roi} Elapsed time: ({datetime.timedelta(seconds=mergetime)})')
                #     print('----------')
                # else:
                #     print (f'Merged watershed dataset {wtd_merge} already created')
                #     print('----------')

                # Begin Zonal Stats
                zstat_start = time.time()
                print(f'Begin zonal statistics min/mean/max std dev for watersheds and catchments in {roi} region')

                # Elevation Zonal statistics  for watersheds
                arcpy.env.snapRaster = elev_rast
                arcpy.env.cellSize = elev_rast
                wtd_elev_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge,
                                                                zone_field = wtd_cur_fields[0],
                                                                in_value_raster = elev_rast,
                                                                out_table = wtd_merge_elev_table_path,
                                                                statistics_type='ALL'
                                                                )
                # Add region identifier field for watershed tables                                                )
                arcpy.AddField_management(wtd_elev_metrics_table,'region',field_type='TEXT')
                # Update region field
                with arcpy.da.UpdateCursor(wtd_elev_metrics_table,'region') as cur:
                    for row in cur:
                        row[0] = roi
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append watershed elev table to list
                wtd_elev_ztables.append(wtd_elev_metrics_table)

                # Elevation zonal statistics for catchments
                arcpy.env.snapRaster = elev_rast
                arcpy.env.cellSize = elev_rast
                cat_elev_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats ,
                                                                zone_field = cat_cur_fields[0],
                                                                in_value_raster = elev_rast,
                                                                out_table = cat_elev_table_path,
                                                                statistics_type='ALL'
                                                                )
                # Add region identifier field for catchment table
                arcpy.AddField_management(cat_elev_metrics_table,'region',field_type='TEXT')

                # Update region field
                with arcpy.da.UpdateCursor(cat_elev_metrics_table,'region') as cur:
                    for row in cur:
                        row[0] = roi
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append catchment elev table to list
                cat_elev_ztables.append(cat_elev_metrics_table)

                # # Aspect Zonal statistics  for watersheds
                # wtd_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge, zone_field ="cat_ID_txt",
                #                                                in_value_raster = asp_rast, out_table = wtd_merge_asp_table_path,
                #                                                statistics_type='ALL')
                # arcpy.AddField_management(wtd_asp_metrics_table, 'region', field_type='TEXT')
                # arcpy.CalculateField_management(wtd_asp_metrics_table, 'region', 'roi')
                # wtd_asp_ztables.append(wtd_asp_metrics_table)

                # # Aspect Zonal statistics for catchments
                # cat_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats, zone_field ="cat_ID_txt",
                #                                                in_value_raster = asp_rast, out_table = cat_asp_table_path,
                #                                                statistics_type='ALL')
                # cat_asp_ztables.append(cat_asp_metrics_table)

                zstat_stop = time.time()
                zstat_time = int (zstat_stop - zstat_start)
                print(f'Zonal Stats for {roi} Elapsed time: ({datetime.timedelta(seconds=zstat_time)})')
                print('----------')

                # Tabulate Area with north grid and catchments/watersheds
                tabarea_start = time.time()
                print(f'Begin tabulate area of north facing cells for watersheds and catchments in {roi} region')
                # Percent North Tabulate area for watersheds
                wtd_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= wtd_merge,
                                                              zone_field= wtd_cur_fields[0],
                                                              in_class_data=nor_rast,
                                                              class_field="Value",
                                                              out_table=wtd_merge_pernorth_table_path
                                                              )
                # Add region and percent north fields
                arcpy.AlterField_management(wtd_per_north_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
                arcpy.AddField_management(wtd_per_north_tabarea, 'region', field_type='TEXT')
                arcpy.AddField_management(wtd_per_north_tabarea, 'wtd_north_per', field_type='Float')
                arcpy.AddField_management(wtd_per_north_tabarea, wtd_cur_fields[0], field_type='TEXT')
                arcpy.AddField_management(wtd_per_north_tabarea, wtd_cur_fields[2], field_type='TEXT')
                wtdnorfields = [f.name for f in arcpy.ListFields(wtd_per_north_tabarea)]
                print (wtdnorfields)
                with arcpy.da.UpdateCursor(wtd_per_north_tabarea, wtdnorfields) as cur:
                    for row in cur:
                        row[4] = roi
                        row[5] = row[3]/(row[3]+row[2])*100
                        row[6] = row[1]
                        row[7] = roi +'_'+ row[1]
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Drop UPPERCASE field form tab area
                arcpy.DeleteField_management(wtd_per_north_tabarea,'CAT_ID_TXT_DEL')
                # Append watershed percent north table to list
                wtd_pernorth_taba_tables.append(wtd_per_north_tabarea)

                # # Percent North Tabulate Area for catchments
                # cat_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= cats, zone_field='cat_ID_con',
                #                                             in_class_data=nor_rast,"Value",
                #                                             out_table=cat_pernorth_table_path)

                # # Add and calculate region identifier field for catchment table
                # arcpy.AlterField_management(cat_per_north_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
                # arcpy.AddField_management(cat_per_north_tabarea, 'region', field_type='TEXT')
                # arcpy.AddField_management(cat_per_north_tabarea, 'cat_north_per', field_type='Float')
                # arcpy.AddField_management(cat_per_north_tabarea, cat_cur_fields[0], field_type='TEXT')
                # arcpy.AddField_management(cat_per_north_tabarea, cat_cur_fields[2], field_type='TEXT')
                # catnorfields = [f.name for f in arcpy.ListFields(cat_per_north_tabarea)]
                # print (catnorfields)
                # with arcpy.da.UpdateCursor(cat_per_north_tabarea,catnorfields) as cur:
                #     for row in cur:
                #         row[4] = roi
                #         row[5] = row[3]/(row[3]+row[2])*100
                #         row[6] = row[1]
                #         row[7] = roi +'_'+ row[1]
                #         # Update
                #         cur.updateRow(row)
                #     del(row)
                # del(cur)
                # Drop UPPERCASE field form tab area
                # arcpy.DeleteField_management(cat_per_north_tabarea,'CAT_ID_TXT_DEL')
                # # Append catchment percent north table to list
                # cat_pernorth_taba_tables.append(cat_per_north_tabarea)
                tabarea_stop = time.time()
                tabarea_time = int (tabarea_stop - tabarea_start)
                print(f'Tabulate area for {roi} Elapsed time: ({datetime.timedelta(seconds=tabarea_time)})')
                print('----------')

            except:
                e = sys.exc_info()[1]
                print(e.args[0])
                arcpy.AddError(e.args[0])

    else:
        print(f'Region {str(roi)} not found in {region}')
# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')



Kodiak using data from D:\GIS_temp\AKSSF\Kodiak folder
Kodiak in ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound'] TauDEM list, using cat_fields ['cat_ID_txt', 'gridcode', 'cat_ID_con'] and watershed fields ['cat_ID_txt', 'cat_ID', 'cat_ID_con']
TauDem data: north raster - D:\Basedata\AKSSF_Basedata\AKSSF_Composite_10m_north_aspect.tif
         aspect raster - D:\Basedata\AKSSF_Basedata\AKSSF_Composite_10m_aspect.tif
      elevation raster - D:\Basedata\AKSSF_Basedata\AKSSF_Composite_10m_extract.tif
----------
cat_ID_txt field already in dataset D:\GIS_temp\AKSSF\Kodiak\Kodiak.gdb\cats_intersect
----------
cat_ID_con field already in dataset D:\GIS_temp\AKSSF\Kodiak\Kodiak.gdb\cats_intersect
----------
Merged watershed dataset wtds_merge found
----------
cat_ID_txt field already in dataset
----------
cat_ID_con field already in dataset D:\GIS_temp\AKSSF\Kodiak\Kodiak.gdb\wtds_merge
----------
 wtd_48267 watershed identified
 wtd_49617 watershed identified
 wtd_50197 watershed identifie

## Drop unnecessary fields and rename as needed from merged tables.
- Create Key value dictionary and use update cursor to rename fields.

In [199]:
# Table names/paths
wtd_per_north_table_out = os.path.join(outgdb, 'AKSSF_wtd_north_per')
cat_elev_table_out = os.path.join(outgdb,'AKSSF_cat_elev')
wtd_elev_table_out = os.path.join(outgdb, 'AKSSF_wtd_elev')

# Merge all regional tables together
wtd_per_north = arcpy.Merge_management(wtd_pernorth_taba_tables, wtd_per_north_table_out)
cat_elev = arcpy.Merge_management(cat_elev_ztables, cat_elev_table_out)
wtd_elev = arcpy.Merge_management(wtd_elev_ztables, wtd_elev_table_out)

In [200]:
#Set up field dictionary
dict = { 'ZONE_CODE': ('cat_elev_ZONE_CODE', 'wtd_elev_ZONE_CODE'),
         'COUNT': ('cat_elev_COUNT', 'wtd_elev_COUNT'),
          'AREA': ('cat_elev_AREA', 'wtd_elev_AREA'),
          'MIN': ('cat_elev_MIN', 'wtd_elev_MIN'),
          'MAX': ('cat_elev_MAX', 'wtd_elev_MAX'),
          'RANGE': ('cat_elev_RANGE', 'wtd_elev_RANGE'),
          'MEAN': ('cat_elev_MEAN', 'wtd_elev_MEAN'),
          'STD': ('cat_elev_STD', 'wtd_elev_STD'),
          'SUM': ('cat_elev_SUM', 'wtd_elev_SUM'),
          'VARIETY': ('cat_elev_VARIETY', 'wtd_elev_VARIETY'),
          'MAJORITY': ('cat_elev_MAJORITY', 'wtd_elev_MAJORITY'),
          'MINORITY': ('cat_elev_MINORITY', 'wtd_elev_MINORITY'),
          'MEDIAN': ('cat_elev_MEDIAN', 'wtd_elev_MEDIAN'),
          'PCT90': ('cat_elev_PCT90', 'wtd_elev_PCT90')
         }

# Rename fields
for field in arcpy.ListFields(wtd_elev):
    keyval = field.name
    if keyval in dict:
        newname = dict[keyval][1]
        newalias = dict[keyval][1]
        print (keyval, newname)
        arcpy.AlterField_management(wtd_elev, keyval, newname, newalias)

for field in arcpy.ListFields(cat_elev):
    keyval = field.name
    if keyval in dict:
        newname = dict[keyval][0]
        newalias = dict[keyval][0]
        print (keyval, newname)
        arcpy.AlterField_management(cat_elev, keyval, newname, newalias)

# for field in arcpy.ListFields(wtd_per_north):
#     keyval = field.name
#     if keyval in dict:
#         newname = dict[keyval][0]
#         newalias = dict[keyval][0]
#         print (keyval, newname)
#         arcpy.AlterField_management(cat_elev, keyval, newname, newalias)


ZONE_CODE wtd_elev_ZONE_CODE
COUNT wtd_elev_COUNT
AREA wtd_elev_AREA
MIN wtd_elev_MIN
MAX wtd_elev_MAX
RANGE wtd_elev_RANGE
MEAN wtd_elev_MEAN
STD wtd_elev_STD
SUM wtd_elev_SUM
VARIETY wtd_elev_VARIETY
MAJORITY wtd_elev_MAJORITY
MINORITY wtd_elev_MINORITY
MEDIAN wtd_elev_MEDIAN
PCT90 wtd_elev_PCT90
ZONE_CODE cat_elev_ZONE_CODE
COUNT cat_elev_COUNT
AREA cat_elev_AREA
MIN cat_elev_MIN
MAX cat_elev_MAX
RANGE cat_elev_RANGE
MEAN cat_elev_MEAN
STD cat_elev_STD
SUM cat_elev_SUM
VARIETY cat_elev_VARIETY
MAJORITY cat_elev_MAJORITY
MINORITY cat_elev_MINORITY
MEDIAN cat_elev_MEDIAN
PCT90 cat_elev_PCT90


## Convert tables to pandas df and explore results
* **Merge tables and export to csv**

In [201]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format # only display 2 decimal places

In [202]:
# Make catchment elev df
cat_df = pd.DataFrame()
cat_field_list = []
for field in arcpy.ListFields(cat_elev):
    cat_field_list.append(field.name)
    print (field.name, field.aliasName)
cat_elev_arr = arcpy.da.TableToNumPyArray(cat_elev,cat_field_list)
cat_df = pd.DataFrame(cat_elev_arr)
cat_df = cat_df.drop(["OBJECTID","cat_elev_ZONE_CODE"],axis=1)
cat_df = cat_df.set_index('cat_ID_txt')
cat_df

OBJECTID OBJECTID
cat_ID_txt cat_ID_txt
cat_elev_ZONE_CODE cat_elev_ZONE_CODE
cat_elev_COUNT cat_elev_COUNT
cat_elev_AREA cat_elev_AREA
cat_elev_MIN cat_elev_MIN
cat_elev_MAX cat_elev_MAX
cat_elev_RANGE cat_elev_RANGE
cat_elev_MEAN cat_elev_MEAN
cat_elev_STD cat_elev_STD
cat_elev_SUM cat_elev_SUM
cat_elev_VARIETY cat_elev_VARIETY
cat_elev_MAJORITY cat_elev_MAJORITY
cat_elev_MINORITY cat_elev_MINORITY
cat_elev_MEDIAN cat_elev_MEDIAN
cat_elev_PCT90 cat_elev_PCT90
region region


,cat_elev_COUNT,cat_elev_AREA,cat_elev_MIN,cat_elev_MAX,cat_elev_RANGE,cat_elev_MEAN,cat_elev_STD,cat_elev_SUM,cat_elev_VARIETY,cat_elev_MAJORITY,cat_elev_MINORITY,cat_elev_MEDIAN,cat_elev_PCT90,region
cat_ID_txt,,,,,,,,,,,,,,
48267,7607.00,760700.00,9,48,39,30.06,8.90,228638.00,40,34,9,32,40,Kodiak
49617,1470.00,147000.00,12,54,42,19.59,9.47,28800.00,43,15,34,16,30,Kodiak
50197,819.00,81900.00,13,35,22,21.07,5.38,17260.00,23,14,33,21,28,Kodiak
64593,2080.00,208000.00,13,75,62,45.99,13.43,95652.00,63,49,14,47,63,Kodiak
72144,657.00,65700.00,38,121,83,80.80,19.10,53086.00,84,74,38,79,106,Kodiak
76954,4168.00,416800.00,29,80,51,41.13,8.84,171422.00,52,33,29,38,54,Kodiak
77794,4689.00,468900.00,15,88,73,27.75,10.86,130100.00,74,21,83,24,38,Kodiak
90346,19618.00,1961800.00,81,638,557,183.12,112.56,3592436.00,546,81,513,140,353,Kodiak
93176,853.00,85300.00,123,195,72,140.28,16.51,119656.00,67,131,181,133,168,Kodiak


In [ ]:
# Make watershed elev df
wtd_df = pd.DataFrame()
wtd_field_list = []
for field in arcpy.ListFields(wtd_elev):
    wtd_field_list.append(field.name)
wtd_elev_arr = arcpy.da.TableToNumPyArray(wtd_elev,wtd_field_list)
wtd_df = pd.DataFrame(wtd_elev_arr)
wtd_df = wtd_df.drop(["OBJECTID","wtd_elev_ZONE_CODE"],axis=1)
wtd_df = wtd_df.set_index('cat_ID_txt')
wtd_df

In [ ]:
# Make watershed north df
wtd_n_df = pd.DataFrame()
wtd_n_field_list = []
for field in arcpy.ListFields(wtd_per_north):
    wtd_n_field_list.append(field.name)
wtd_n_arr = arcpy.da.TableToNumPyArray(wtd_per_north,wtd_n_field_list)
wtd_n_df = pd.DataFrame(wtd_n_arr)
wtd_n_df = wtd_n_df.drop("OBJECTID",axis=1)
wtd_n_df = wtd_n_df.set_index('cat_ID_txt')
wtd_n_df

In [ ]:
landmet_merge_df = pd.merge(wtd_df,cat_df,on='cat_ID_txt', how='outer')

In [ ]:
dfs = [wtd_df,cat_df]
df_final = reduce(lambda left,right: pd.merge(left,right,on='cat_ID_txt'), dfs)

In [ ]:
df_final